# CPSC532S Assignment 3:  RNNs for Language Modeling

In [86]:
from collections import Counter
from collections import defaultdict
from gensim.models import Word2Vec
from random import random
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from torch import nn
from torch.autograd import Variable

import numpy as np
import torch
import torch.nn.functional as F
import json
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

# Data Acquisition


The goal of this assignment is to translate English to Pig Latin. For this assignment, you must download the data and extract it into `data/`. The dataset contains four files, each containing a single caption on each line. There are two files for training (English vs Pig Latin) and two files for validation. We should have 20,000 sentences (one sentence per image in Assignment 2) in the training captions and 500 sentences in the validation captions (five sentences per image in Assignment 2).

In [87]:
# from google.colab import drive

# drive.mount('/data')

# Load the data into memory.
mscoco_train = json.load(open("./content/train_captions.json"))
mscoco_val  = json.load(open("./content/val_captions.json"))

mscoco_piglatin_train = json.load(open('./content/piglatin_train_captions.json'))
mscoco_piglatin_val  = json.load(open('./content/piglatin_val_captions.json'))

train_sentences = [entry['caption'] for entry in mscoco_train['annotations']]
train_sentences = train_sentences[:5000]
val_sentences = [entry['caption'] for entry in mscoco_val['annotations']]

piglatin_train_sentences = [entry['caption'] for entry in mscoco_piglatin_train['annotations']]
piglatin_train_sentences = piglatin_train_sentences[:5000]
piglatin_val_sentences = [entry['caption'] for entry in mscoco_piglatin_val['annotations']]

print(len(train_sentences))
print(len(val_sentences))
print(len(piglatin_train_sentences))
print(len(piglatin_val_sentences))
print(train_sentences[0])
print(piglatin_train_sentences[0])
print(val_sentences[0])
print(piglatin_val_sentences[0])

5000
500
5000
500
A very clean and well decorated empty bathroom
Away eryvay eanclay andway ellway ecoratedday emptyway athroombay
Set of bananas hanging off of a banana tree.
Etsay ofway ananasbay anginghay offway ofway away ananabay eetray.


In [88]:
import os 
cwd = os.getcwd()
print(cwd)

/Users/xuanchen/Desktop/532S/assignment/Assignment3


# Preprocessing

The code provided below creates word embeddings for you to use. After creating the vocabulary, we construct both one-hot embeddings and word2vec embeddings. 


In [89]:
import nltk
nltk.download('punkt')

sentences = train_sentences
piglatin_sentences = piglatin_train_sentences

# Lower-case the sentence, tokenize them and add <SOS> and <EOS> tokens
sentences = [["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] for sentence in sentences]
piglatin_sentences = [["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] for sentence in piglatin_sentences]

# Create the vocabulary. Note that we add an <UNK> token to represent words not in our vocabulary.
vocabularySize = 2000
word_counts = Counter([word for sentence in sentences for word in sentence])
piglatin_word_counts = Counter([word for sentence in piglatin_sentences for word in sentence])
word_counts = word_counts + piglatin_word_counts
vocabulary = ["<UNK>"] + [e[0] for e in word_counts.most_common(vocabularySize-1)]
word2index = {word:index for index,word in enumerate(vocabulary)}

# Build the one hot embeddings
one_hot_embeddings = np.eye(vocabularySize)


# Build the word2vec embeddings
wordEncodingSize = 300
filtered_sentences = [[word for word in sentence if word in word2index] for sentence in sentences]
piglatin_filtered_sentences = [[word for word in sentence if word in word2index] for sentence in piglatin_sentences]
all_filtered_sentences = filtered_sentences + piglatin_filtered_sentences
w2v = Word2Vec(all_filtered_sentences, min_count=0, vector_size=wordEncodingSize)
w2v_embeddings = np.concatenate((np.zeros((1, wordEncodingSize)), w2v.wv.vectors))

# Define the max sequence length to be the longest sentence in the training data. 
maxSequenceLength = max([len(sentence) for sentence in sentences])
piglatin_maxSequenceLength = max([len(sentence) for sentence in piglatin_sentences])

if piglatin_maxSequenceLength > maxSequenceLength:
    maxSequenceLength = piglatin_maxSequenceLength


[nltk_data] Downloading package punkt to /Users/xuanchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Utilities functions


Please look through the functions provided below carefully, as you will need to use all of them at some point in your assignment.

In [90]:
def preprocess_numberize(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into list of numbers (denoting the index into the vocabulary).
    """
    tokenized = word_tokenize(sentence.lower())
        
    # Add the <SOS>/<EOS> tokens and numberize (all unknown words are represented as <UNK>).
    tokenized = ["<SOS>"] + tokenized + ["<EOS>"]
    numberized = [word2index.get(word, 0) for word in tokenized]
    
    return numberized

def preprocess_one_hot(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into a numpy array of one-hot vectors.
    """
    numberized = preprocess_numberize(sentence)
    
    # Represent each word as it's one-hot embedding
    one_hot_embedded = one_hot_embeddings[numberized]
    
    return one_hot_embedded

def preprocess_word2vec(sentence):
    """
    Given a sentence, in the form of a string, this function will preprocess it
    into a numpy array of word2vec embeddings.
    """
    numberized = preprocess_numberize(sentence)
    
    # Represent each word as it's one-hot embedding
    w2v_embedded = w2v_embeddings[numberized]
    
    return w2v_embedded

def compute_bleu(reference_sentence, predicted_sentence):
    """
    Given a reference sentence, and a predicted sentence, compute the BLEU similary between them.
    """
    reference_tokenized = word_tokenize(reference_sentence.lower())
    predicted_tokenized = word_tokenize(predicted_sentence.lower())
    return sentence_bleu([reference_tokenized], predicted_tokenized)

%matplotlib inline
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words.split(' ') +['<EOS>'])

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

score1 = compute_bleu("<SOS>" + train_sentences[0], "<SOS>" + train_sentences[0])
score2 = compute_bleu("<SOS>" + train_sentences[0], "<SOS>" + train_sentences[5])

print('BLEU score distnace between \n  "' + train_sentences[0] + '" \nand\n  "'+ train_sentences[0] + '" \nis: ' + str(score1) +'\n\n')
print('BLEU score distnace between \n  "' + train_sentences[0] + '" \nand\n  "'+ train_sentences[5] + '" \nis: ' + str(score2) +'\n\n')


BLEU score distnace between 
  "A very clean and well decorated empty bathroom" 
and
  "A very clean and well decorated empty bathroom" 
is: 1.0


BLEU score distnace between 
  "A very clean and well decorated empty bathroom" 
and
  "A few people sit on a dim transportation system. " 
is: 0.1933853138176172




#Part 1: Encoder-Decoder Language Translation with Teacher-Forcing


## 1.1 Building a Language Decoder

We now implement a language decoder. For now, we will have the decoder take a single training sample at a time (as opposed to batching). For our purposes, we will also avoid defining the embeddings as part of the model and instead pass in embedded inputs. While this is sometimes useful, as it learns/tunes the embeddings, we avoid doing it for the sake of simplicity and speed.

Remember to use LSTM hidden units!

In [91]:
class DecoderLSTM(nn.Module):
    def __init__(self):
        super(DecoderLSTM, self).__init__()
        
        self.hidden_dim = 300
        wordEncodingSize = 2000
        
        # Your code goes here (~4 lines or less)
        self.lstm = nn.LSTM(wordEncodingSize, self.hidden_dim)
        self.linear = nn.Linear(self.hidden_dim, wordEncodingSize)
        self.softmax = nn.LogSoftmax(dim=1)

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_dim)


    def init_cell(self):
        return torch.zeros(1, 1, self.hidden_dim)

    def forward(self, input_sentence, hidden, cell):
        output, (hidden, cell) = self.lstm(input_sentence, (hidden, cell))
        output = self.softmax(self.linear(output[0]))
        return output, hidden, cell


## 1.2.  Building Language Encoder

We now build a language encoder, which will encode an input word by word, and ultimately output a hidden state that we can then be used by our decoder.

In [92]:
class EncoderLSTM(nn.Module):
    # Your code goes here
    def __init__(self):
        super(EncoderLSTM, self).__init__()
        
        self.hidden_dim = 300
        wordEncodingSize = 2000
        
        # Your code goes here (~3 lines)
        self.lstm = nn.LSTM(input_size=wordEncodingSize, hidden_size=self.hidden_dim)
       
    def init_hidden(self):
        # Your code goes here (1 line)
        return torch.zeros(1, 1, self.hidden_dim)
    
    def init_cell(self):
        # Your code goes here (1 line)
        return torch.zeros(1, 1, self.hidden_dim)

    def forward(self, input_sentence, hidden, cell):
        # Your code goes here (~2 lines of code)
        output, (hidden, cell) = self.lstm(input_sentence, (hidden, cell))
        
        return output, hidden, cell
            

In [93]:
input_tensor = torch.tensor(
        preprocess_numberize("A very clean and well decorated empty bathroom"),
        dtype=torch.long
    ).view(-1, 1)
encoder = EncoderLSTM()
encoder_hidden = encoder.init_hidden()
encoder_cell = encoder.init_cell()
input_sentence_one_hot_embeddings = one_hot_embeddings[input_tensor]
print(input_sentence_one_hot_embeddings.shape)
input_length = input_sentence_one_hot_embeddings.shape[0]

# for ei in range(input_length):
#   encoder_input = torch.tensor(input_sentence_one_hot_embeddings[ei,:], dtype=torch.float).view(1,1,-1)
#   encoder_output, encoder_hidden, encoder_cell = encoder(encoder_input, encoder_hidden, encoder_cell)
# print(encoder_output.shape)

(10, 1, 2000)


In [94]:
test = torch.zeros(maxSequenceLength, 300)
print(test.shape)
decoder_input = torch.tensor([preprocess_numberize("<SOS>")])
print(decoder_input.shape)

torch.Size([38, 300])
torch.Size([1, 5])


In [95]:
decoder = DecoderLSTM()
decoder_hidden = encoder_hidden
decoder_cell = encoder_cell
decoder_input = torch.tensor([word2index.get("<SOS>", 0)],dtype=torch.long).view(-1, 1)
decoder_input = one_hot_embeddings[decoder_input]
decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

for di in range(input_length):
  decoder_output, decoder_hidden, decoder_cell = decoder(
      decoder_input, decoder_hidden, decoder_cell
  )
print(decoder_output.shape)
topv, topi = decoder_output.topk(1)
decoder_input = topi.squeeze().detach()
decoder_input = one_hot_embeddings[decoder_input]
decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)
print(topi)
print(decoder_input.shape)
test = torch.tensor([word2index.get("<EOS>", 0)],dtype=torch.long).view(-1, 1)
print(test)

torch.Size([1, 2000])
tensor([[85]])
torch.Size([1, 1, 2000])
tensor([[2]])


## 1.3. Connecting Encoder to Decoder and Train End-to-End and Train with Teacher Forcing

We now connect our newly created encoder with our decoder, to train an end-to-end seq2seq architecture. 

For the purposes of Part 1, the only interaction between the encoder and the decoder is that the *last hidden state of the encoder is used as the initial hidden state of the decoder*. This will be different for Part 2 and 3 where we will extend this punction.

In [162]:
from os import device_encoding


def train(input_sentence, output_sentence, encoder,
          decoder, encoder_optimizer,
          decoder_optimizer, 
          criterion, 
          teacher_forcing_ratio = 1,
          decoderType = "LSTM",
          embeddings = one_hot_embeddings): 
    """
    Given a single training sample, go through a single step of training.
    """
    use_teacher_forcing = True if np.random.rand() < teacher_forcing_ratio else False
    # Your code goes here
    encoder_hidden = encoder.init_hidden()
    encoder_cell = encoder.init_cell()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_tensor = torch.tensor(
        [word2index.get(word, 0) for word in input_sentence],
        dtype=torch.long
    ).view(-1, 1)

    input_sentence_one_hot_embeddings = embeddings[input_tensor]

    output_tensor = torch.tensor(
        [word2index.get(word, 0) for word in output_sentence],
        dtype=torch.long
    ).view(-1, 1)

    output_sentence_one_hot_embeddings = embeddings[output_tensor]

    input_length = input_sentence_one_hot_embeddings.shape[0]
    output_length = output_sentence_one_hot_embeddings.shape[0]
    
    encoder_hidden_states = torch.zeros(maxSequenceLength, encoder.hidden_dim)

    # encoder_outputs = torch.zeros(maxSequenceLength, encoder.hidden_dim)

    loss = 0

    for ei in range(input_length):
        encoder_input = torch.tensor(input_sentence_one_hot_embeddings[ei,:], dtype=torch.float).view(1,1,-1)
        encoder_output, encoder_hidden, encoder_cell = encoder(encoder_input, encoder_hidden, encoder_cell)
        encoder_hidden_states[ei,:] = encoder_hidden[0]
    
    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell
    decoder_input = torch.tensor([word2index.get("<SOS>", 0)],dtype=torch.long).view(-1, 1)
    decoder_input = embeddings[decoder_input]
    decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

    EOS = torch.tensor([word2index.get("<EOS>", 0)],dtype=torch.long).view(-1, 1)
    EOS = embeddings[EOS]
    EOS = torch.tensor(EOS, dtype=torch.float).view(1,1,-1)
    
    output_len = len(output_sentence)
    if decoderType == "Transformer":
        full_sentence_decoder_input = torch.zeros(output_len, vocabularySize)
        full_sentence_decoder_input[0,:] = decoder_input.unsqueeze(0)
        full_sentence_target_output = torch.zeros(output_len, dtype=torch.int64)


    # print(decoder_input.shape)
    # print(decoder_hidden.shape)
    # print(decoder_cell.shape)

    if decoderType == "LSTM": 
        if use_teacher_forcing: 
            # Your code goes here
            for di in range(output_length):
                try:
                    decoder_output, decoder_hidden, decoder_cell = decoder(
                        decoder_input, decoder_hidden, decoder_cell
                    )
                except:
                    print(decoder_input.shape)
                    print(decoder_hidden.shape)
                    print(decoder_cell.shape)
                    print(di)
                    print('use')
                    raise
                output_tensor = output_sentence_one_hot_embeddings[di,:]
                decoder_input = torch.tensor(output_tensor, dtype=torch.float).view(1,1,-1)
                output_tensor = torch.tensor(output_tensor, dtype=torch.float).view(1,-1)
                loss += criterion(decoder_output, output_tensor)
            
        elif not use_teacher_forcing:
            # Your code goes here
            for di in range(output_length):
                try:
                    decoder_output, decoder_hidden, decoder_cell = decoder(
                        decoder_input, decoder_hidden, decoder_cell
                    )
                except:
                    print(decoder_input.shape)
                    print(decoder_hidden.shape)
                    print(decoder_cell.shape)
                    print('not use')
                    raise
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()
                decoder_input = embeddings[decoder_input]
                decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

                output_tensor = output_sentence_one_hot_embeddings[di,:]
                output_tensor = torch.tensor(output_tensor, dtype=torch.float).view(1,-1)
                loss += criterion(decoder_output, output_tensor)
                if topi.squeeze().item() == word2index["<EOS>"]:
                      break
                # if decoder_input.item() == EOS:
                #       break
                
    if decoderType == "AttentionLSTM":
        for i in range(output_length):
            decoder_output, decoder_hidden, decoder_cell, attention_weights = decoder(decoder_input, decoder_hidden, decoder_cell, encoder_hidden_states)
            next_word = output_sentence[i]
            next_word_indx = word2index[next_word]
            decoder_word_input = embeddings[next_word_indx]
            target_output = torch.LongTensor(np.nonzero(decoder_word_input)[0])
            
            if use_teacher_forcing: 
                decoder_input = torch.FloatTensor(decoder_word_input)
            elif not use_teacher_forcing:
                topv, topi = log_softmax(decoder_output[0]).topk(1)
                decoder_input = torch.FloatTensor(embeddings[topi.squeeze().detach().item()])
            
            if decoderType != "Transformer":     
                loss += criterion(decoder_output[0], target_output)
            
            if not use_teacher_forcing:
                if topi.squeeze().item() == word2index["<EOS>"]:
                    break

    if decoderType == "Transformer":
        for i in range(1, output_length):
            next_word = output_sentence[i]
            next_word_indx = word2index[next_word]
            decoder_word_input = embeddings[next_word_indx]
            target_output = torch.LongTensor(np.nonzero(decoder_word_input)[0])

            # Your code goes here (you can assume transformer uses teacher_forcing_ratio = 1)
            full_sentence_decoder_input[i,:] = torch.FloatTensor(decoder_word_input)
            full_sentence_target_output[i-1] = torch.LongTensor(np.nonzero(decoder_word_input)[0])
         

    if decoderType == "Transformer":
        full_sentence_decoder_output, self_attn, encoder_decoder_attn = decoder(full_sentence_decoder_input, decoder_hidden, decoder_cell, encoder_hidden_states)
        loss += criterion(full_sentence_decoder_output[0], full_sentence_target_output) 

    # Your code goes here
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / output_length

In [97]:
# Train the model and monitor the loss. Remember to use Adam optimizer and CrossEntropyLoss
encoder = EncoderLSTM()
decoder = DecoderLSTM()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.0005) 
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.0005) 
criterion = nn.CrossEntropyLoss()
epochs = 5

print("Start training end-to-end network ......")
for epoch in range(epochs):
    epoch_loss=[]
    count=0
    for id, sentence in enumerate(filtered_sentences):
        target_variable = piglatin_filtered_sentences[id]
        # print(sentence)
        # print(target_variable)
        loss = train(sentence, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, teacher_forcing_ratio = 1, decoderType="LSTM")

        count = count+1
        if count%500==0:
            print("Single sentence Loss (epoch %d) : %f" % (epoch, loss))
        epoch_loss.append(loss)
        
    print("Loss (epoch %d) : %f" % (epoch, np.sum(epoch_loss)/len(filtered_sentences))) 

Start training end-to-end network ......
Single sentence Loss (epoch 0) : 4.028410
Single sentence Loss (epoch 0) : 2.726545
Single sentence Loss (epoch 0) : 3.118297
Single sentence Loss (epoch 0) : 3.166363
Single sentence Loss (epoch 0) : 3.310811
Single sentence Loss (epoch 0) : 2.493892
Single sentence Loss (epoch 0) : 4.538890
Single sentence Loss (epoch 0) : 4.550961
Single sentence Loss (epoch 0) : 4.187765
Single sentence Loss (epoch 0) : 2.731255
Loss (epoch 0) : 3.386638
Single sentence Loss (epoch 1) : 3.213170
Single sentence Loss (epoch 1) : 1.801395
Single sentence Loss (epoch 1) : 2.900025
Single sentence Loss (epoch 1) : 2.930545
Single sentence Loss (epoch 1) : 2.381593
Single sentence Loss (epoch 1) : 2.086346
Single sentence Loss (epoch 1) : 3.730067
Single sentence Loss (epoch 1) : 3.884600
Single sentence Loss (epoch 1) : 3.335449
Single sentence Loss (epoch 1) : 2.443882
Loss (epoch 1) : 2.761695
Single sentence Loss (epoch 2) : 2.939102
Single sentence Loss (epo

## 1.4. Building Language Decoder MAP Inference

We now define a method to perform inference with our decoder. 

In [167]:
def inference(sentence, encoder, decoder, decoderType="LSTM", embeddings=one_hot_embeddings, max_length=maxSequenceLength):
    input_length = len(sentence)
    decoder_attentions = 0

    # Initialize encoder & decoder
    encoder_hidden = encoder.init_hidden()
    encoder_cell = encoder.init_cell()
    # decoder_hidden = decoder.init_hidden()
    # decoder_cell = decoder.init_hidden()
    encoder_hidden_states = torch.zeros(max_length, encoder.hidden_dim)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad() 

    for ei in range(1,input_length):
        # Iteratively run the encoder 
        # 1. Get the current word index
        wordIdx = word2index[sentence[ei]]
        # 2. Convert to a 1-hot encoding
        encoder_input = torch.Tensor(embeddings[wordIdx]).view(1,1,-1)
        # 3. Run one step of the encoder
        encoder_output, encoder_hidden, encoder_cell = encoder(encoder_input, encoder_hidden, encoder_cell)
        # 4. Save the encoder hidden states for future processing
        encoder_hidden_states[ei,:] = encoder_hidden[0]

    # Set the initial hidden and cell state of the RNN decoder to the last 
    # hidden and cell state of the encoder
    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    # Start the decoding with <SOS> token
    decoder_input = torch.tensor([word2index.get("<SOS>", 0)],dtype=torch.long).view(-1, 1)
    decoder_input = embeddings[decoder_input]
    decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

    EOS = torch.tensor([word2index.get("<EOS>", 0)],dtype=torch.long).view(-1, 1)

    word_list = []
    decoder_attentions = torch.zeros(max_length, max_length)

    # Iterate up to the max_length of output
    for i in range(max_length):
        if decoderType == "LSTM": 
            # Run the simple decoder
            decoder_output, decoder_hidden, decoder_cell = decoder(
                decoder_input, decoder_hidden, decoder_cell
            )
            topv, topi = decoder_output.topk(1)
            if topi == EOS:
                word_list.append('<EOS>')
                break
            else:
                word_list.append(vocabulary[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = embeddings[decoder_input]
            decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

        if decoderType == "AttentionLSTM":
            decoder_output, decoder_hidden, decoder_cell, attention_weights = decoder(decoder_input, decoder_hidden, decoder_cell, encoder_hidden_states)
            decoder_attentions[i] = attention_weights.detach().cpu()[0,:,0]
            
            topv, topi = decoder_output.data.topk(1)
            if topi == EOS:
                word_list.append('<EOS>')
                break
            else:
                word_list.append(vocabulary[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = embeddings[decoder_input]
            decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

        if decoderType == "Transformer":
            decoder_output, encoder_decoder_attn, _ = decoder(decoder_input, decoder_hidden, decoder_cell, encoder_hidden_states)
            decoder_attentions = encoder_decoder_attn
            decoder_output = decoder_output[:,-1:,:]
            topv, topi = decoder_output[0][-1].data.topk(1)
            if topi.item() == EOS:
                word_list.append('<EOS>')
                break
            else:
                word_list.append(vocabulary[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = embeddings[decoder_input]
            decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)
    
    res_list = ""
    for i in range(len(word_list)):
        if word_list[i] == "<EOS>" or word_list[i] == "<SOS>":
            continue
        res_list = res_list + " " + word_list[i]

    return res_list, decoder_attentions

In [113]:
# Lets test it 
sentence = "A very clean and well decorated empty bathroom" 
input_sentence = ["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] 
output_sentence, _ = inference(input_sentence, encoder, decoder, decoderType="LSTM")

print("English: " + sentence)
print("Pig Latin: " + output_sentence)

English: A very clean and well decorated empty bathroom
Pig Latin:  away ofway itewhay andway away argelay aintray .


## 1.5. Building Language Decoder Sampling Inference

We now modify the inference method to sample from the distribution outputted by the LSTM rather than taking the most probable word.

It might be useful to take a look at the output of your model and (depending on your implementation) modify it so that the outputs sum to 1. 

In [122]:
def sampling_inference(sentence, encoder, decoder, decoderType="LSTM", embeddings=one_hot_embeddings, max_length=maxSequenceLength):
    # input_tensor = torch.Tensor(preprocess_one_hot(sentence))
    input_length = len(sentence)

    # Initialize encoder & decoder 
    encoder_hidden = encoder.init_hidden()
    encoder_cell = encoder.init_cell()
    decoder_hidden = decoder.init_hidden()
    decoder_cell = decoder.init_hidden()
    encoder_hidden_states = torch.zeros(max_length, encoder.hidden_dim)
    decoder_attentions = torch.zeros(max_length, max_length)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    for ei in range(1,input_length):
        # Iteratively run the encoder 
        # 1. Get the current word index
        # 2. Convert to a 1-hot encoding
        encoder_input = torch.Tensor(embeddings[word2index[sentence[ei]]]).view(1,1,-1)
        # 3. Run one step of the encoder
        encoder_output, encoder_hidden, encoder_cell = encoder(encoder_input, encoder_hidden, encoder_cell)
        # 4. Save the encoder hidden states for future processing
        encoder_hidden_states[ei,:] = encoder_hidden[0]

    # Set the initial hidden and cell state of the RNN decoder to the last 
    # hidden and cell state of the encoder
    decoder_hidden = encoder_hidden
    decoder_cell = encoder_cell

    # Start the decoding with <SOS> token
    decoder_input = torch.tensor([word2index.get("<SOS>", 0)],dtype=torch.long).view(-1, 1)
    decoder_input = embeddings[decoder_input]
    decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

    EOS = torch.tensor([word2index.get("<EOS>", 0)],dtype=torch.long).view(-1, 1)

    word_list = []
    decoder_attentions = torch.zeros(max_length, max_length)

    # Iterate up to the max_length of output
    for i in range(max_length):
        if decoderType == "LSTM": 
            # Run the simple decoder 
            decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS:
                word_list.append('<EOS>')
                break
            else:
                word_list.append(vocabulary[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = embeddings[decoder_input]
            decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)

        if decoderType == "AttentionLSTM":
            decoder_output, decoder_hidden, decoder_cell, decoder_attention = decoder(
                decoder_input, decoder_hidden, decoder_cell, encoder_output
            )
            decoder_attentions[i] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi == EOS:
              word_list.append('<EOS>')
              break
            else:
              word_list.append(vocabulary[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = embeddings[decoder_input]
            decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)
            

        if decoderType == "Transformer":
            decoder_output, encoder_attention_weights, self_attention_weights = decoder(decoder_input,encoder_output.unsqueeze())
            topv, topi = decoder_output[0][-1].data.topk(1)
            if topi.item() == EOS:
              word_list.append('<EOS>')
              break
            else:
              word_list.append(vocabulary[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = embeddings[decoder_input]
            decoder_input = torch.tensor(decoder_input, dtype=torch.float).view(1,1,-1)
    
    res_list = ""
    for i in range(len(word_list)):
        if word_list[i] == "<EOS>" or word_list[i] == "<SOS>":
            continue
        else: res_list = res_list + " " + word_list[i]


    return res_list, decoder_attentions
    

In [123]:
# Lets test it 
sentence = "A very clean and well decorated empty bathroom" 
input_sentence = ["<SOS>"] + word_tokenize(sentence.lower()) + ["<EOS>"] 

print("English: " + sentence)

for i in range(5):
    output_sentence, _ = sampling_inference(input_sentence, encoder, decoder, decoderType="LSTM")
    print("Pig Latin: " + output_sentence)

English: A very clean and well decorated empty bathroom
Pig Latin:  away ueblay andway itewhay usbay isway arkedpay
Pig Latin:  away ueblay andway itewhay usbay isway arkedpay
Pig Latin:  away ueblay andway itewhay usbay isway arkedpay
Pig Latin:  away ueblay andway itewhay usbay isway arkedpay
Pig Latin:  away ueblay andway itewhay usbay isway arkedpay


## 1.6. Testing 

We must now define a method that allows us to do inference using the seq2seq architecture. We then run the 500 validation captions through this method, and ultimately compare the **reference** and **generated** sentences using our **BLEU** similarity score method defined above, to identify the average BLEU score.

In [116]:
# Perform inference for all validation sequences and report the average BLEU score
avg_score=[]

# iterate over the validation set 
for idx, input_sentence in enumerate(val_sentences):
    input_tok = ["<SOS>"] + word_tokenize(input_sentence.lower()) + ["<EOS>"] 
    input_good = [word for word in input_tok if word in word2index]
    
    output_sentence, _ = inference(input_good, encoder, decoder, decoderType="LSTM")
    target_sentence = piglatin_val_sentences[idx]
    score = compute_bleu(target_sentence, output_sentence)
    avg_score.append(score)
    if idx < 5 :
        print('BLEU score distance between \n  "' + target_sentence + '" \nand\n  "'+ output_sentence + '" \n is: ' + str(score) +'\n\n')

final_score = np.sum(avg_score)/len(val_sentences)
print("Average BLEU score : %f" % (final_score)) 


# EXPECTED < Average BLUE score (ArgMAX inference): 0.464803 > 
# EXPECTED < Average BLUE score (sampling inference): 0.477803 > 

BLEU score distance between 
  "Etsay ofway ananasbay anginghay offway ofway away ananabay eetray." 
and
  " iewvay ofway away aintray ationstay ithway away argelay itewhay ." 
 is: 6.373704167435469e-155


BLEU score distance between 
  "Otway unchesbay ofway eengray ananasbay onway ananabay eestray." 
and
  " otway ofway irdsbay inway ethay ofway away ." 
 is: 1.258141043412406e-231


BLEU score distance between 
  "Anymay alendarscay andway unchesbay ofway ananasbay anginghay onway away allway." 
and
  " eethray irdsbay areway azinggray inway ethay assgray ofway ." 
 is: 1.0016022933125248e-231


BLEU score distance between 
  "Ustersclay ofway ananasbay andway icturespay anginghay onway away allway." 
and
  " eoplepay andway away iraffegay alkingway inway ethay assgray ." 
 is: 1.2387197655613557e-231


BLEU score distance between 
  "otway ogsday atthay ooklay otay ebay ightingfay oneway anotherway" 
and
  " otway irdsbay areway ittingsay onway ethay assgray ." 
 is: 9.55980462367

## 1.7. Experiment with Teacher Forcing

Redo steps 1.3 and 1.6 with teacher_forcing_ratio = 0.9 and 0.8. Comment on the results, speed of convergence and the quality of results. Note that in most real scenarious the teacher forcing is actually annealed; starting with teacher forcing = 1.

In [119]:
# Your code goes here
encoder = EncoderLSTM()
decoder = DecoderLSTM()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.0005) 
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.0005) 
criterion = nn.CrossEntropyLoss()
epochs = 5

for epoch in range(epochs):
    epoch_loss=[]
    count=0
    for idx, input_sentence in enumerate(filtered_sentences):
        output_sentence = piglatin_filtered_sentences[idx]
        loss = train(input_sentence, output_sentence, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, teacher_forcing_ratio = 0.9, decoderType="LSTM") 
        count = count + 1
        if count % 500 == 0:
            print("Single sentence Loss (epoch %d) : %f" % (epoch, loss))
        epoch_loss.append(loss)
        
    print("Loss (epoch %d) : %f" % (epoch, np.sum(epoch_loss)/len(filtered_sentences)))

Single sentence Loss (epoch 0) : 3.804468
Single sentence Loss (epoch 0) : 2.774093
Single sentence Loss (epoch 0) : 3.157748
Single sentence Loss (epoch 0) : 3.333501
Single sentence Loss (epoch 0) : 3.362679
Single sentence Loss (epoch 0) : 2.586376
Single sentence Loss (epoch 0) : 4.486127
Single sentence Loss (epoch 0) : 4.485689
Single sentence Loss (epoch 0) : 4.302087
Single sentence Loss (epoch 0) : 2.670315
Loss (epoch 0) : 3.476956
Single sentence Loss (epoch 1) : 3.332968
Single sentence Loss (epoch 1) : 1.918672
Single sentence Loss (epoch 1) : 2.864294
Single sentence Loss (epoch 1) : 3.111577
Single sentence Loss (epoch 1) : 2.549426
Single sentence Loss (epoch 1) : 2.194239
Single sentence Loss (epoch 1) : 3.804628
Single sentence Loss (epoch 1) : 4.061269
Single sentence Loss (epoch 1) : 3.836316
Single sentence Loss (epoch 1) : 2.363047
Loss (epoch 1) : 2.989311
Single sentence Loss (epoch 2) : 3.064690
Single sentence Loss (epoch 2) : 1.553361
Single sentence Loss (ep

In [124]:
# Perform inference for all validation sequences and report the average BLEU score
avg_score=[]

# iterate over the validation set 
for idx, input_sentence in enumerate(val_sentences):
    input_tok = ["<SOS>"] + word_tokenize(input_sentence.lower()) + ["<EOS>"]
    input_good = [word for word in input_tok if word in word2index]
    
    output_sentence, _ = inference(input_good, encoder, decoder, decoderType="LSTM")
    target_sentence = piglatin_val_sentences[idx]
    score = compute_bleu(target_sentence, output_sentence)
    avg_score.append(score)
    if idx < 5 :
        print('BLEU score distance between \n  "' + target_sentence + '" \nand\n  "'+ output_sentence + '" \n is: ' + str(score) +'\n\n')

final_score = np.sum(avg_score)/len(val_sentences)
print("Average BLEU score : %f" % (final_score)) 

BLEU score distance between 
  "Etsay ofway ananasbay anginghay offway ofway away ananabay eetray." 
and
  " ofway away aintray oinggay oughthray ethay ofway away eetray ." 
 is: 8.612150057732663e-155


BLEU score distance between 
  "Otway unchesbay ofway eengray ananasbay onway ananabay eestray." 
and
  " otway iraffesgay alkingway inway ethay assgray ieldfay ." 
 is: 1.1368587676511996e-231


BLEU score distance between 
  "Anymay alendarscay andway unchesbay ofway ananasbay anginghay onway away allway." 
and
  " eoplepay andway away iraffegay andingstay inway ethay assgray ." 
 is: 1.1084551487393798e-231


BLEU score distance between 
  "Ustersclay ofway ananasbay andway icturespay anginghay onway away allway." 
and
  " eoplepay onway away enchbay inway ethay iddlemay ofway away ." 
 is: 6.8489908526642754e-155


BLEU score distance between 
  "otway ogsday atthay ooklay otay ebay ightingfay oneway anotherway" 
and
  " otway irdsbay areway onway away itycay eetstray ." 
 is: 9.55

## 1.8. Encoding as Generic Feature Representation

We now use the final hidden state of our encoder, to identify the nearest neighbor amongst the training sentences for each sentence in our validation data.

It would be effective to first define a method that would generate all of the hidden states and store these hidden states **on the CPU**, and then loop over the generated hidden states to identify/output the nearest neighbors.

In [125]:
from os import device_encoding

def final_encoder_hidden(sentence, embeddings=one_hot_embeddings):
    # Your code goes here
    encoder_hidden = encoder.init_hidden()
    encoder_cell = encoder.init_cell()
    numerized = preprocess_numberize(sentence)
    sentence_one_hot_embeddings = embeddings[numerized]
    input_length = sentence_one_hot_embeddings.shape[0]
    for ei in range(input_length):
        encoder_input = torch.tensor(sentence_one_hot_embeddings[ei,:], dtype=torch.float).view(1,1,-1)
        encoder_output, encoder_hidden, encoder_cell = encoder(encoder_input , encoder_hidden, encoder_cell)
        
    return encoder_output.view(-1)
# Now run all training data and validation data to store hidden states
# Your code goes here
val_contexts = []
train_contexts = []
for sentence in val_sentences:
    val_contexts.append(final_encoder_hidden(sentence).cpu().data.numpy())
np.save(open('validation_vectors', 'wb+'), val_contexts)
    
for sentence in train_sentences:
    train_contexts.append(final_encoder_hidden(sentence).cpu().data.numpy())
np.save(open('training_vectors', 'wb+'), train_contexts)

val_contexts = np.load('validation_vectors')
train_contexts = np.load('training_vectors')

In [126]:
# Now get nearest neighbors and print
from sklearn.metrics.pairwise import euclidean_distances
distances = euclidean_distances(val_contexts, train_contexts)
min_distances = np.argmin(distances, axis=1)
print(min_distances.shape)
print(min_distances.shape)
i = 0
for index in min_distances[:10]:
    print(val_sentences[i])
    i = i+1
    print(train_sentences[index])

(500,)
(500,)
Set of bananas hanging off of a banana tree.
Collection of flower pots of various sizes, filled with various flowers.
Two bunches of green bananas on banana trees.
Two sliced of toasted angel food cake sitting on a white plate.
Many calendars and bunches of bananas hanging on a wall.
Some animals and a bicycle that are by a house.
Clusters of bananas and pictures hanging on a wall.
landscape of buildings and a statue in a city
two dogs that look to be fighting one another
Two discarded toilets face each other on the side of the road
Two dogs fighting with one on his back on the ground
Two children poised with a fire hydrant on the sidewalk.
Bunches of green bananas hanging down from trees.
landscape of buildings and a statue in a city
Two dogs have a playful fight with one another.
Two settler ladies working in a modern day kitchen.
Banana trees with large  hanging bunches of bananas.
Stoplight with red light on top of monitor
There are some green bananas hanging in bunch

# Part 2: Attention LSTM Decoder

## 2.1. Implementing Additive Attention

In [127]:
class AdditiveAttention(nn.Module):
    def __init__(self, hidden_size):
        super(AdditiveAttention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_network = nn.Sequential(
                                    nn.Linear(hidden_size*2, hidden_size),
                                    nn.ReLU(),
                                    nn.Linear(hidden_size, 1)
                                 )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, queries, keys, values):
        """The forward pass of the additive attention mechanism.

        Arguments:
            queries: The current decoder hidden state. (batch_size x hidden_size)
            keys: The encoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)
            values: The encoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)

        Returns:
            context: weighted average of the values (batch_size x 1 x hidden_size)
            attention_weights: Normalized attention weights for each encoder hidden state. (batch_size x 1 x seq_len)

            The attention_weights must be a softmax weighting over the seq_len annotations.
        """

        # ------------
        # Your code goes here
        # ------------
        batch_size = queries.shape[0]
        expanded_queries = queries.unsqueeze(1).expand_as(keys)
        concat_inputs = torch.cat((expanded_queries, keys), 2)
        unnormalized_attention = self.attention_network(concat_inputs) 
        attention_weights = self.softmax(unnormalized_attention)
        context = torch.bmm(attention_weights.transpose(1, 2), values.unsqueeze(0))

        return context, attention_weights

## 2.2. Attention Decoder

In [128]:
class AttentionDecoder(nn.Module):
    def __init__(self):
        super(AttentionDecoder, self).__init__()

        self.hidden_dim = 300
        wordEncodingSize = 2000
        self.dropout_p = 0.1
        self.linear_input = nn.Linear(wordEncodingSize, self.hidden_dim)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_dim*2, self.hidden_dim)
        self.attention = AdditiveAttention(hidden_size=self.hidden_dim)
        self.linear = nn.Linear(self.hidden_dim, vocabularySize)
        self.hidden = self.init_hidden()
        self.cell = self.init_cell()

    def init_hidden(self):    
        return torch.randn(1,1, self.hidden_dim)

    def init_cell(self):
        return torch.randn(1,1, self.hidden_dim)

    def forward(self, input_sentence, hidden, cell, encoder_hidden_states):
        # ------------
        # Your code goes here
        # ------------
        embed = self.dropout(self.linear_input(input_sentence.view(1,-1)))
        context, attention_weights = self.attention(embed, encoder_hidden_states.unsqueeze(0), encoder_hidden_states) 
        embed_and_context = torch.cat([embed.unsqueeze(0),context], 2) 
        output, (hidden, cell) = self.lstm(embed_and_context, (hidden, cell))
        output = self.linear(output)

        return output, hidden , cell, attention_weights


## 2.3. Training Attention Decoder

Note that you will need to modify the train() procedure for Part 1 to handles the AttentionLSTM.

In [129]:
# Train the model and monitor the loss. Remember to use Adam optimizer and CrossEntropyLoss
encoder = EncoderLSTM()
decoder = AttentionDecoder()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.0005) 
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.0005) 
criterion = nn.CrossEntropyLoss()
epochs = 5

print("Start training end to end network ......")
for epoch in range(epochs):
    epoch_loss=[]
    count=0
    for id, sentence in enumerate(filtered_sentences):
        target_variable = piglatin_filtered_sentences[id]
        loss = train(sentence, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, teacher_forcing_ratio = 1, decoderType="AttentionLSTM")
        count = count+1
        if count%500==0:
            print("Single sentence Loss (epoch %d) : %f" % (epoch, loss))
        epoch_loss.append(loss)
        
    print("Loss (epoch %d) : %f" % (epoch, np.sum(epoch_loss)/len(filtered_sentences))) 

Start training end to end network ......
Single sentence Loss (epoch 0) : 4.010912
Single sentence Loss (epoch 0) : 2.348166
Single sentence Loss (epoch 0) : 3.094016
Single sentence Loss (epoch 0) : 2.861800
Single sentence Loss (epoch 0) : 2.664646
Single sentence Loss (epoch 0) : 2.606734
Single sentence Loss (epoch 0) : 4.368237
Single sentence Loss (epoch 0) : 4.211634
Single sentence Loss (epoch 0) : 4.111131
Single sentence Loss (epoch 0) : 2.682067
Loss (epoch 0) : 3.173156
Single sentence Loss (epoch 1) : 3.265067
Single sentence Loss (epoch 1) : 1.577080
Single sentence Loss (epoch 1) : 2.777534
Single sentence Loss (epoch 1) : 2.641054
Single sentence Loss (epoch 1) : 2.312160
Single sentence Loss (epoch 1) : 2.086554
Single sentence Loss (epoch 1) : 3.881687
Single sentence Loss (epoch 1) : 3.465160


KeyboardInterrupt: 

## 2.4. Testing Attention Decoder
Note that you will need to modify the inference() procedure for Part 1 to handle Attention LSTM

In [132]:
# Perform inference for all validation sequences and report the average BLEU score
avg_score=[]

# iterate over the validation set 
for idx, input_sentence in enumerate(val_sentences):
    input_tok = ["<SOS>"] + word_tokenize(input_sentence.lower()) + ["<EOS>"] 
    input_good = [word for word in input_tok if word in word2index]
    
    output_sentence, _ = inference(input_good, encoder, decoder, decoderType="AttentionLSTM")
    target_sentence = piglatin_val_sentences[idx]
    score = compute_bleu(target_sentence, output_sentence)
    avg_score.append(score)
    if idx < 5 :
        print('BLEU score distance between \n  "' + target_sentence + '" \nand\n  "'+ output_sentence + '" \n is: ' + str(score) +'\n\n')

final_score = np.sum(avg_score)/len(val_sentences)
print("Average BLEU score : %f" % (final_score))

# EXPECTED < Average BLUE score (ArgMAX inference): 0.739589 >

/Users/xuanchen/miniforge3/envs/pixplot/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/xuanchen/miniforge3/envs/pixplot/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/xuanchen/miniforge3/envs/pixplot/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it cont

BLEU score distance between 
  "Etsay ofway ananasbay anginghay offway ofway away ananabay eetray." 
and
  " erethay areway away usbay inway away assygray areaway ." 
 is: 1.1193096620723278e-231


BLEU score distance between 
  "Otway unchesbay ofway eengray ananasbay onway ananabay eestray." 
and
  " erethay areway away affictray ightlay inway ethay ackgroundbay" 
 is: 0


BLEU score distance between 
  "Anymay alendarscay andway unchesbay ofway ananasbay anginghay onway away allway." 
and
  " erethay areway away usbay inway away ieldfay ." 
 is: 8.853864984883467e-232


BLEU score distance between 
  "Ustersclay ofway ananasbay andway icturespay anginghay onway away allway." 
and
  " erethay isway away usbay inway away ieldfay ." 
 is: 1.0032743411283238e-231


BLEU score distance between 
  "otway ogsday atthay ooklay otay ebay ightingfay oneway anotherway" 
and
  " erethay areway away usbay inway ethay ackgroundbay ." 
 is: 0


Average BLEU score : 0.001363


## 2.5. Visualize Attention for Attention Decoder

In [133]:
# Your code goes here
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words.split(' ') +['<EOS>'])

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

score1 = compute_bleu("<SOS>" + train_sentences[0], "<SOS>" + train_sentences[0])
score2 = compute_bleu("<SOS>" + train_sentences[0], "<SOS>" + train_sentences[5])

print('BLEU score distnace between \n  "' + train_sentences[0] + '" \nand\n  "'+ train_sentences[0] + '" \nis: ' + str(score1) +'\n\n')
print('BLEU score distnace between \n  "' + train_sentences[0] + '" \nand\n  "'+ train_sentences[5] + '" \nis: ' + str(score2) +'\n\n')

BLEU score distnace between 
  "A very clean and well decorated empty bathroom" 
and
  "A very clean and well decorated empty bathroom" 
is: 1.0


BLEU score distnace between 
  "A very clean and well decorated empty bathroom" 
and
  "A few people sit on a dim transportation system. " 
is: 0.1933853138176172




# Part 3: Transformer Decoder

## 3.1 Implement Scaled Dot Attention

In [ ]:
class ScaledDotAttention(nn.Module):
    def __init__(self, hidden_size):
        super(ScaledDotAttention, self).__init__()

        self.hidden_size = hidden_size

        self.Q = nn.Linear(hidden_size, hidden_size)
        self.K = nn.Linear(hidden_size, hidden_size)
        self.V = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=1)
        self.scaling_factor = torch.rsqrt(torch.tensor(self.hidden_size, dtype= torch.float))

    def forward(self, queries, keys, values):
        """The forward pass of the scaled dot attention mechanism.

        Arguments:
            queries: The current decoder hidden state, 2D or 3D tensor. (batch_size x (k) x hidden_size)
            keys: The encoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)
            values: The encoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)

        Returns:
            context: weighted average of the values (batch_size x k x hidden_size)
            attention_weights: Normalized attention weights for each encoder hidden state. (batch_size x seq_len x 1)

            The output must be a softmax weighting over the seq_len annotations.
        """

        # ------------
        # Your code goes here
        # ------------
        batch_size = queries.size(0)
        if queries.dim() != 3:
            queries = torch.unsqueeze(queries, dim = 1)
        q = self.Q(queries)
        k = self.K(keys)
        k = torch.transpose(k, 1, 2)
        v = self.V(values)
        unnormalized_attention = torch.bmm(q,k) * self.scaling_factor
        attention_weights = self.softmax(unnormalized_attention)
        context = torch.bmm(attention_weights, v)

        return context, attention_weights


## 3.2. Implement Causal Scaled Dot Attention

The implementation should be nearly identical to the one above, but with mask.

In [137]:
class CausalScaledDotAttention(nn.Module):
    def __init__(self, hidden_size):
        super(CausalScaledDotAttention, self).__init__()

        self.hidden_size = hidden_size
        self.neg_inf = torch.tensor(-1e7)

        self.Q = nn.Linear(hidden_size, hidden_size)
        self.K = nn.Linear(hidden_size, hidden_size)
        self.V = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=1)
        self.scaling_factor = torch.rsqrt(torch.tensor(self.hidden_size, dtype= torch.float))

    def forward(self, queries, keys, values):
        """The forward pass of the scaled dot attention mechanism.

        NOTES:
            batch_size = 1

        Arguments:
            queries: The current decoder hidden state, 2D or 3D tensor. (batch_size x (k) x hidden_size)
                In training k = maxSequenceLength or length of the GT ourput sequence
                In testing k = length of currently decoded sub-sequence
            keys: The decoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)
            values: The decoder hidden states for each step of the input sequence. (batch_size x seq_len x hidden_size)

        Returns:
            context: weighted average of the values (batch_size x k x hidden_size)
            attention_weights: Normalized attention weights for each encoder hidden state. (batch_size x seq_len x 1)

            The output must be a softmax weighting over the seq_len annotations.
        """

        # ------------
        # Your code goes here
        # ------------
        batch_size = queries.size(0)

        k_value = queries.size(1)
        seq_len = keys.size(1)

        if queries.dim() != 3:
            queries = torch.unsqueeze(queries, dim = 1)
        q = self.Q(queries)
        k = self.K(keys)
        k = torch.transpose(k, 1, 2)
        v = self.V(values)
        unnormalized_attention = torch.bmm(q,k) * self.scaling_factor
        mask = torch.tril(torch.ones(batch_size, k_value, seq_len))
        unnormalized_attention[mask == 0] = self.neg_inf
        attention_weights = self.softmax(unnormalized_attention)
        context = torch.bmm(attention_weights, v)

        return context, attention_weights

## 3.3. Implement Transformer Decoder

In [148]:
class TransformerDecoder(nn.Module):
    def __init__(self):
        super(TransformerDecoder, self).__init__()

        self.hidden_dim = 300
        wordEncodingSize = 2000
        self.dropout_p = 0.1
        self.num_layers = 3
        self.linear_input = nn.Linear(wordEncodingSize, self.hidden_dim)
        self.dropout = nn.Dropout(self.dropout_p)
        
        self.self_attentions = nn.ModuleList([CausalScaledDotAttention(
                                    hidden_size=self.hidden_dim, 
                                 ) for i in range(self.num_layers)])
        self.encoder_attentions = nn.ModuleList([ScaledDotAttention(
                                    hidden_size=self.hidden_dim, 
                                 ) for i in range(self.num_layers)])
        self.attention_mlps = nn.ModuleList([nn.Sequential(
                                    nn.Linear(self.hidden_dim, self.hidden_dim),
                                    nn.ReLU(),
                                 ) for i in range(self.num_layers)])

        self.linear = nn.Linear(self.hidden_dim, vocabularySize)

    def forward(self, input_sentence, hidden, cell, annotations):
        embed = self.dropout(self.linear_input(input_sentence)).unsqueeze(0)
        
        encoder_attention_weights_list = []
        self_attention_weights_list = []
        contexts = embed
        batch_size, seq_len, hidden_size = contexts.size()

        for i in range(self.num_layers):
            new_contexts, self_attention_weights = self.self_attentions[i](contexts, contexts, contexts)
            residual_contexts = contexts + new_contexts
            new_contexts, encoder_attention_weights = self.encoder_attentions[i](residual_contexts,annotations.unsqueeze(0), annotations.unsqueeze(0))
            residual_contexts = residual_contexts + new_contexts
            new_contexts = self.attention_mlps[i](residual_contexts.view(-1, self.hidden_dim)).view(batch_size, seq_len, self.hidden_dim)
            contexts = residual_contexts + new_contexts
            
            encoder_attention_weights_list.append(encoder_attention_weights)
            self_attention_weights_list.append(self_attention_weights)            
        
        output = self.linear(contexts)
        encoder_attention_weights = torch.stack(encoder_attention_weights_list)
        self_attention_weights = torch.stack(self_attention_weights_list)
        
        return output, encoder_attention_weights, self_attention_weights

## 3.4. Training Transformer Decoder

Note that you will need to modify the train() procedure for Part 1 to handle the Transformer.

In [149]:
# Train the model and monitor the loss. Remember to use Adam optimizer and CrossEntropyLoss
encoder = EncoderLSTM()
decoder = TransformerDecoder()
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=0.0005) 
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=0.0005) 
criterion = nn.CrossEntropyLoss()
epochs = 5

print("Start training end to end network ......")
for epoch in range(epochs):
    epoch_loss=[]
    count=0
    for id, sentence in enumerate(filtered_sentences):
        target_variable = piglatin_filtered_sentences[id]
        loss = train(sentence, target_variable, encoder, decoder,encoder_optimizer,decoder_optimizer, criterion, teacher_forcing_ratio = 1, decoderType="Transformer")
        count = count+1
        if count%500==0:
            print("Single sentence Loss (epoch %d) : %f" % (epoch, loss))
        epoch_loss.append(loss)
        
    print("Loss (epoch %d) : %f" % (epoch, np.sum(epoch_loss)/len(filtered_sentences))) 

Start training end to end network ......
Single sentence Loss (epoch 0) : 0.419194
Single sentence Loss (epoch 0) : 0.163597
Single sentence Loss (epoch 0) : 0.220472
Single sentence Loss (epoch 0) : 0.178288
Single sentence Loss (epoch 0) : 0.196344
Single sentence Loss (epoch 0) : 0.147067
Single sentence Loss (epoch 0) : 0.257912
Single sentence Loss (epoch 0) : 0.342656
Single sentence Loss (epoch 0) : 0.196589
Single sentence Loss (epoch 0) : 0.057299
Loss (epoch 0) : 0.199598
Single sentence Loss (epoch 1) : 0.167105
Single sentence Loss (epoch 1) : 0.004150
Single sentence Loss (epoch 1) : 0.036919
Single sentence Loss (epoch 1) : 0.044097
Single sentence Loss (epoch 1) : 0.000772
Single sentence Loss (epoch 1) : 0.047090
Single sentence Loss (epoch 1) : 0.071687
Single sentence Loss (epoch 1) : 0.078573
Single sentence Loss (epoch 1) : 0.013199
Single sentence Loss (epoch 1) : 0.040677
Loss (epoch 1) : 0.047187
Single sentence Loss (epoch 2) : 0.071531
Single sentence Loss (epo

## 3.5. Testing Transformer Decoder
Note that you will need to modify the inference() procedure for Part 1 to handle Transformer

In [168]:
# Perform inference for all validation sequences and report the average BLEU score
avg_score=[]

# iterate over the validation set 
for idx, input_sentence in enumerate(val_sentences): 
    input_tok = ["<SOS>"] + word_tokenize(input_sentence.lower()) + ["<EOS>"] 
    input_good = [word for word in input_tok if word in word2index]
    
    output_sentence, _ = inference(input_good, encoder, decoder, decoderType="Transformer")
    target_sentence = "<SOS>"+output_sentence
    score = compute_bleu("<SOS>"+target_sentence[0], "<SOS>"+output_sentence)
    avg_score.append(score)
    if idx < 10 :
        print('BLEU score distance between \n  "' + target_sentence + '" \nand\n  "'+ output_sentence + '" \n is: ' + str(score) +'\n\n')

final_score = np.sum(avg_score)/len(val_sentences)
print("Average BLEU score : %f" % (final_score))  

ValueError: too many values to unpack (expected 3)

## 3.6 Visualizing Attention for Transformer Decoder

Note that since we have multiple attention layers, there will be one attention to be visualized per layer. 

In [169]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words.split(' ') +['<EOS>'])

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

score1 = compute_bleu("<SOS>" + train_sentences[0], "<SOS>" + train_sentences[0])
score2 = compute_bleu("<SOS>" + train_sentences[0], "<SOS>" + train_sentences[5])

print('BLEU score distnace between \n  "' + train_sentences[0] + '" \nand\n  "'+ train_sentences[0] + '" \nis: ' + str(score1) +'\n\n')
print('BLEU score distnace between \n  "' + train_sentences[0] + '" \nand\n  "'+ train_sentences[5] + '" \nis: ' + str(score2) +'\n\n')

BLEU score distnace between 
  "A very clean and well decorated empty bathroom" 
and
  "A very clean and well decorated empty bathroom" 
is: 1.0


BLEU score distnace between 
  "A very clean and well decorated empty bathroom" 
and
  "A few people sit on a dim transportation system. " 
is: 0.1933853138176172




# 4. Effectiveness of word2vec

As an option, you may repeat one of the models above by modifying the code to use word2vec embedding for the input English sentences

In [ ]:
# Your code goes here

: 